In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import warnings
warnings.filterwarnings('ignore')

from touls import import_csv_RAW_data

In [2]:
CLIENT_FILE = "../data/RAW/clients.csv"
VENTES_FILE = "../data/RAW/ventes.csv"
PRODUITS_FILE = "../data/RAW/produits.csv"

clients_df, ventes_df, produits_df = import_csv_RAW_data(CLIENT_FILE,VENTES_FILE,PRODUITS_FILE)

# Check & clean data
## client shoud be unique, not null

In [3]:
if clients_df.duplicated(subset = "client_id" ,keep=False).sum() > 0:
        print ("There is duplicated values on Id")

if clients_df.isnull().values.any() : 
    print("There is null values")

## Product should have unique id, no null value, positive price

In [4]:
if produits_df.duplicated(subset = "id_prod" ,keep=False).sum() > 0:
        print ("There is duplicated values on Id")

if clients_df.isnull().values.any() : 
    print("There is null values")
if (produits_df['price'] < 0).sum() >= 0:
    print ("There is product with negative price: \n",produits_df[produits_df['price'] < 0])
#produits_df = produits_df[produits_df['price'] > 0]

There is product with negative price: 
     id_prod  price  categ
731     T_0   -1.0      0


## Ventes should have valid date, no null value

In [5]:
ventes_df["date_match"] = pd.to_datetime(ventes_df["date"], format="%Y-%m-%d %H:%M:%S.%f", errors = 'coerce')
ventes_df_error = ventes_df[ventes_df['date_match'].isnull()]
print("There is",len(ventes_df_error) , "value with unvalid date\n", ventes_df[ventes_df['date_match'].isnull()][["id_prod","date","session_id","client_id"]])

There is 200 value with unvalid date
        id_prod                             date session_id client_id
1431       T_0  test_2021-03-01 02:30:02.237420        s_0      ct_1
2365       T_0  test_2021-03-01 02:30:02.237446        s_0      ct_1
2895       T_0  test_2021-03-01 02:30:02.237414        s_0      ct_1
5955       T_0  test_2021-03-01 02:30:02.237441        s_0      ct_0
7283       T_0  test_2021-03-01 02:30:02.237434        s_0      ct_1
...        ...                              ...        ...       ...
332594     T_0  test_2021-03-01 02:30:02.237445        s_0      ct_0
332705     T_0  test_2021-03-01 02:30:02.237423        s_0      ct_1
332730     T_0  test_2021-03-01 02:30:02.237421        s_0      ct_1
333442     T_0  test_2021-03-01 02:30:02.237431        s_0      ct_1
335279     T_0  test_2021-03-01 02:30:02.237430        s_0      ct_0

[200 rows x 4 columns]


### 'product' : T_0, 'client' : ct_0, ct_1 and 'ventes' : test are only for test
### they are dropped from the database

In [6]:
produits_df_clean = produits_df[produits_df['price'] >= 0]
clients_df_clean = clients_df[(clients_df['client_id']!='ct_1') & (clients_df['client_id']!='ct_0')]
clients_df_clean['age'] = date.today().year - clients_df_clean['birth']
ventes_df_clean = ventes_df[~ventes_df['date_match'].isnull()]
ventes_df_clean["date"] =  pd.to_datetime(ventes_df_clean["date"], format="%Y-%m-%d %H:%M:%S.%f", errors = 'raise')
ventes_df_clean = ventes_df_clean.drop(['date_match'],axis = 1)

In [7]:
ventes_produits = ventes_df_clean.merge(produits_df_clean, left_on='id_prod', right_on='id_prod',how='outer')
ventes_produits_erros = ventes_produits[(ventes_produits['price'].isnull().values) | (ventes_produits['date'].isnull().values)]
prod_notExist = ventes_produits_erros[ventes_produits_erros['price'].isnull()]
prod_notselled = ventes_produits_erros[ventes_produits_erros['date'].isnull()]
print("There are ",len(prod_notExist)," products not exicted in database \n", "must add them: \n", prod_notExist)
print("There are ",len(prod_notselled)," products never selled \n", "must promote them!: \n", prod_notselled)

There are  103  products not exicted in database 
 must add them: 
        id_prod                       date session_id client_id  price  categ
307791  0_2245 2021-06-17 03:03:12.668129    s_49705    c_1533    NaN    NaN
307792  0_2245 2021-06-16 05:53:01.627491    s_49323    c_7954    NaN    NaN
307793  0_2245 2021-11-24 17:35:59.911427   s_124474    c_5120    NaN    NaN
307794  0_2245 2022-02-28 18:08:49.875709   s_172304    c_4964    NaN    NaN
307795  0_2245 2021-03-01 00:09:29.301897        s_3     c_580    NaN    NaN
...        ...                        ...        ...       ...    ...    ...
307889  0_2245 2021-04-06 19:59:19.462288    s_16936    c_4167    NaN    NaN
307890  0_2245 2021-03-30 23:29:02.347672    s_13738    c_7790    NaN    NaN
307891  0_2245 2021-12-03 14:14:40.444177   s_128815    c_6189    NaN    NaN
307892  0_2245 2021-04-27 18:58:47.703374    s_26624    c_1595    NaN    NaN
307893  0_2245 2021-05-01 03:35:03.146305    s_28235    c_5714    NaN    NaN

[103 ro

In [8]:
ventes_clients = ventes_df_clean.merge(clients_df_clean, left_on='client_id', right_on='client_id',how='outer')
ventes_clients_erros = ventes_clients[ventes_clients.isnull().values]
ventes_clients_erros=ventes_clients_erros[~ventes_clients_erros.duplicated(keep='first')]
print("There are ",len(ventes_clients_erros), " clients who never buyed a product! \n","Must send them ads!\n", ventes_clients_erros[['client_id','sex','age']])

There are  21  clients who never buyed a product! 
 Must send them ads!
        client_id sex  age
336816    c_8253   f   20
336817    c_3789   f   24
336818    c_4406   f   23
336819    c_2706   f   54
336820    c_3443   m   62
336821    c_4447   m   65
336822    c_3017   f   29
336823    c_4086   f   29
336824    c_6930   m   17
336825    c_4358   m   22
336826    c_8381   f   56
336827    c_1223   m   58
336828    c_6862   f   19
336829    c_5245   f   17
336830    c_5223   m   18
336831    c_6735   m   17
336832     c_862   f   65
336833    c_7584   f   61
336834      c_90   m   20
336835     c_587   m   28
336836    c_3526   m   65


# Create a new merged database to use it for analysis

In [9]:
ventes_produits_clean = ventes_df_clean.merge(produits_df_clean, left_on='id_prod', right_on='id_prod',how='inner')
ventes_produits_client_clean = ventes_produits_clean.merge(clients_df_clean, left_on='client_id', right_on='client_id',how='inner')
clients_df_clean.to_pickle("../data/CURATED/clients.pkl")
produits_df_clean.to_pickle("../data/CURATED/produits.pkl")
ventes_df_clean.to_pickle("../data/CURATED/ventes.pkl")
ventes_produits_client_clean.to_pickle("../data/CURATED/merged.pkl")